# Teil 3: Fortgeschrittene Remote Execution Werkzeuge

Im letzten Abschnitt wurde ein einfaches Model mittels Federated Learning trainiert. Mit `.send()` und `.get()` wurde das Model zu den Trainingsdaten gesendet, ein Update durchgeführt und anschließend zurück gebracht. Doch am Ende des Beispiels wurde klar, dass dies nicht ausreicht um die Privatsphäre der Beteiligten zu schützen. Deshalb wollen wir nun mehrere Gradienten mitteln **bevor** wir sie mit `.get()` zurückholen. Auf diese Weise erhalten wir keinerlei Einsicht in den exakten Gradienten (und schützen so die Privatsphäre des Einzelnen!!!).

Um dies zu erreichen, brauchen wir einige weitere Bausteine:

- verwenden eines Pointers um einen Tensor zu einem anderen Helfer zu senden

Wenn wir gerade dabei sind, werden zusätzlich noch einige weitere fortgeschrittene Tensor Functionen behandelt, welche dieses wie auch spätere Beispiele vereinfachen.

Autoren:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

Übersetzer:
- Jan Moritz Behnken - Github: [@JMBehnken](https://github.com/JMBehnken)

In [ ]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

# Abschnitt 3.1 - Pointer zu Pointer

Wie Sie wissen, können PointerTensor Objekte genau wie normale Tensoren behandelt werden. Um genau zu sein, sie sind _so ähnlich zu Tensoren_, dass wir Pointer **zu** anderen Pointern haben können. Sehen Sie selbst!

In [ ]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [ ]:
# this is a local tensor
x = torch.tensor([1,2,3,4])
x

In [ ]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

In [ ]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

### Was ist hier genau geschehen?

Im vorherigen Beispiel wurde ein Tensor `x` erstellt. Beim Senden des Tensors zu Bob wurde ein Pointer (`x_ptr`) auf dem lokalen Rechner erstellt.

Beim Aufrufen von `x_ptr.send(alice)` wurde **dieser Pointer gesendet** an Alice.

Anmerkung:  
Hierbei wurden die Daten NICHT bewegt! Einzig allein der Pointer auf die Daten wurde verschoben!!

In [ ]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

In [ ]:
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
alice._objects

In [ ]:
# and we can use .get() to get x_ptr back from Alice

x_ptr = pointer_to_x_ptr.get()
x_ptr

In [ ]:
# and then we can use x_ptr to get x back from Bob!

x = x_ptr.get()
x

### Arithmetik mit Pointer -> Pointer -> Daten Objekt

Genau wie mit normalen Pointern können beliebige PyTorch Operationen auf den Tensoren angewendet werden. 

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
y.get().get()

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
p2p2x.get().get()

In [ ]:
bob._objects

In [ ]:
alice._objects

# Abschnitt 3.2 - Pointer Ketten Operationen

Im letzten Abschnitt wurden die `.send()` und `.get()` Befehle immer direkt auf dem Tensor der lokalen Maschine ausgeführt. Doch bei einer Kette von Pointern kann es manchmal wünschenswert sein Befehle wie `.get()` oder `.send()` auf dem **letzten** Pointer der Kette aufzurufen (wie z. B. das Senden von Daten zwischen Helfern). Um dies zu erreichen, können vertrauliche und speziell dafür entwickelte Funktionen verwendet werden.

Diese Operationen sind:

- `my_pointer2pointer.move(another_worker)`

In [ ]:
# x is now a pointer to the data which lives on Bob's machine
x = torch.tensor([1,2,3,4,5]).send(bob)

In [ ]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

In [ ]:
x = x.move(alice)

In [ ]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

In [ ]:
x

Hervorragend! Nun sind Sie bereit um ferngesteuert zusammen mit einem vertrauenswürdigen Aggregator **Gradienten zu mitteln**!

# Herzlichen Glückwunsch!!! - Zeit, der Community beizutreten! 

Herzlichen Glückwunsch zum Abschluss dieses Notebook-Tutorials! Wenn es Ihnen gefallen hat und Sie sich der Bewegung zur Wahrung der Privatsphäre, zum dezentralisiertenen Besitz von KI und der KI-Lieferkette (Daten) anschließen möchten, können Sie dies auf folgende Weise tun! 

### PySyft auf GitHub einen Stern geben! 

Der einfachste Weg, unserer Community zu helfen, besteht darin, die GitHub-Repos mit Sternen auszuzeichnen! Dies hilft, das Bewusstsein für die coolen Tools zu schärfen, die wir bauen. 

- [Gib PySyft einen Stern](https://github.com/OpenMined/PySyft)

### Mach mit bei Slack! 

Der beste Weg, um über die neuesten Entwicklungen auf dem Laufenden zu bleiben, ist, sich unserer Community anzuschließen! Sie können dies tun, indem Sie das Formular unter [http://slack.openmined.org](http://slack.openmined.org) ausfüllen.

### Treten Sie einem Code-Projekt bei! 

Der beste Weg, um zu unserer Community beizutragen, besteht darin, Entwickler zu werden! Sie können jederzeit zur PySyft GitHub Issues-Seite gehen und nach "Projects" filtern. Dies zeigt Ihnen alle Top-Level-Tickets und gibt einen Überblick darüber, an welchen Projekten Sie teilnehmen können! Wenn Sie nicht an einem Projekt teilnehmen möchten, aber ein wenig programmieren möchten, können Sie auch nach weiteren "einmaligen" Miniprojekten suchen, indem Sie nach GitHub-Problemen suchen, die als "good first issue" gekennzeichnet sind. 

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Spenden

Wenn Sie keine Zeit haben, zu unserer Codebase beizutragen, aber dennoch Unterstützung leisten möchten, können Sie auch Unterstützer unseres Open Collective werden. Alle Spenden fließen in unser Webhosting und andere Community-Ausgaben wie Hackathons und Meetups! 

 - [OpenMined's Open Collective Page](https://opencollective.com/openmined)